# PyMogwai Live Demo!

## Initialization
First, we need to import the mogwai library

In [1]:
from mogwai.core.traversal import MogwaiGraphTraversalSource
from mogwai.core.steps.statics import *
from mogwai.parser import graphml_to_mogwaigraph

Then, load the graph!

In [2]:
airroutes_path = "tests/documents/air-routes-latest.graphml"
graph = graphml_to_mogwaigraph(airroutes_path, node_label_key="labelV", 
            edge_label_key="labelE", node_name_key="desc")

Finally, create a traversal source

In [3]:
g = MogwaiGraphTraversalSource(graph)

## Let's go on a holiday!
We want to go on holiday, where can we go from germany?

In [5]:
query = g.V().has_label("country").has_name("Germany")\
        .out("contains")\
        .out("route")\
        .in_("contains")\
        .has_label("country").name().dedup().to_list()
print("Query:",query.print_query())
res = query.run()
print("Result:", res[:10], "total: ", len(res))

Query: V -> Contains -> Has -> Out -> Out -> In -> Contains -> Values -> Dedup -> ToList
Result: ['United States', 'Puerto Rico', 'Canada', 'United Kingdom', 'France', 'Finland', 'Japan', 'Singapore', 'United Arab Emirates', 'India'] total:  110


Okay, that's a bit much. Let's see what's the farthest we can get without a layover

In [16]:
query = g.V().has_label("country").has_name("Germany")\
        .out("contains")\
        .outE("route").as_("e")\
        .inV()\
        .in_("contains").has_label("country").as_('dest')\
        .select('e').properties('dist').max_()\
        .select('dest').by('name').next()
print("Result:", query.run())

Result: Argentina


Well, that's nice. Let's go there! But from which airport are we departing?

In [17]:
query = g.V().has_label("country").has_name("Germany")\
            .out("contains").as_('start')\
            .outE("route").as_("e")\
            .inV()\
            .in_("contains").has_label("country").as_('dest')\
            .select('e').properties('dist').as_('dist').max_().\
            select('start', 'dest', 'dist').to_list().by('name')
res = query.run()
print("Result:",res)

Result: [['Frankfurt am Main', 'Argentina', 7141]]


Hmm, that's a bit far away from Aachen. Let's see where we can go from a nearby airport

In [18]:
g.V().has_label("airport").within("city", ["Brussels", "Maastricht", "Aachen", "Dusseldorf"]).as_('start')\
            .outE("route").as_("e")\
            .inV()\
            .in_("contains").has_label("country").as_('dest')\
            .order(desc=True).by(select('e').properties('dist'))\
            .limit(5)\
            .select('start', 'dest').to_list().by('name').run()

[['Dusseldorf International Airport', 'Singapore'],
 ['Brussels Airport', 'Japan'],
 ['Brussels South Charleroi Airport', 'Hong Kong'],
 ['Brussels Airport', 'Hong Kong'],
 ['Brussels Airport', 'China']]

Those flights are really long...
I guess we have to accept layovers then. Where can we go within three hops if we want to depart from Maastricht Aachen Airport? We don't want a flight longer than 2000 miles.

In [21]:
g.V().has_label('airport').has_property("city", "Maastricht").as_('start')\
    .repeat(
        outE('route').as_('e')\
        .inV().filter_(
            select('e').properties('dist').is_(lte(2000))
        ).simple_path()
    ).times(3).emit().as_('dest')\
    .path().as_('p')\
    .count(Scope.local).order(asc=True).as_('length')\
    .select('dest').dedup()\
    .select('dest', 'length').to_list().by('city').run()

[['Alicante', 3],
 ['Alghero', 3],
 ['Trapani', 3],
 ['Girona', 3],
 ['Catania', 3],
 ['Kerkyra Island', 3],
 ['Heraklion', 3],
 ['Tenerife Island', 3],
 ['Bari', 3],
 ['Porto', 3],
 ['Faro', 3],
 ['Zakynthos', 3],
 ['Antalya', 3],
 ['Palma De Mallorca', 3],
 ['Istanbul', 3],
 ['Luqa', 3],
 ['Kos Island', 3],
 ['Geneva', 5],
 ['Liverpool', 5],
 ['Nottingham', 5],
 ['London', 5],
 ['Shannon', 5],
 ['Manchester', 5],
 ['Brussels', 5],
 ['Zurich', 5],
 ['Kalymnos Island', 5],
 ['Barcelona', 5],
 ['Rome', 5],
 ['Dublin', 5],
 ['Paris', 5],
 ['London', 5],
 ['London', 5],
 ['Berlin', 5],
 ['Paris', 5],
 ['Madrid', 5],
 ['Nice', 5],
 ['Luxembourg', 5],
 ['Lyon', 5],
 ['Cork', 5],
 ['Basle', 5],
 ['Sevilla', 5],
 ['Bristol', 5],
 ['Leeds', 5],
 ['Malaga', 5],
 ['Milan', 5],
 ['Gran Canaria', 5],
 ['Lisbon', 5],
 ['Doncaster', 5],
 ['Nantes', 5],
 ['Bologna', 5],
 ['Pisa', 5],
 ['Naples', 5],
 ['Stuttgart', 5],
 ['Bilbao', 5],
 ['London', 5],
 ['Rostock', 5],
 ['Kaunas', 5],
 ['Ostrava', 5],
 